<a href="https://colab.research.google.com/github/AnjaliKrishna123/Project/blob/main/Nutrition_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import required libraries
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
df = pd.read_csv('mock_data.csv')
properties = list(df.columns.values)
properties.remove('Category')

y = df['Category']
c = [p for p in range(len(y))]
    
m_data = pd.DataFrame(np.zeros((len(y),len(properties))),columns=properties,index=c)

for i in range(df.shape[0]):
    for p in properties:
        data = df.iloc[i][p]
        if type(data) == str:
            print("We got a string, should not happen")
            data = np.float64(data.replace(',',''))
            if np.isnan(data):
                #outcome = df.iloc[i]['Category']
                cleanData(m_data,p,outcome,i)
            else:
                m_data.iloc[i][p] = data
                print("-Added to m_data")
        elif np.isnan(data):
            #outcome = df.iloc[i]['Category']
            pass
        else:
            m_data.iloc[i][p] = data

X = m_data[properties]
X.head

<bound method NDFrame.head of       BMI  Weight (Kg)  ...  Folate (mcg)  Diabetic
0    17.0         50.0  ...         390.0       0.0
1    18.0         54.0  ...         380.0       0.0
2    16.0         56.0  ...         397.0       0.0
3    17.0         57.0  ...         364.0       0.0
4    18.0         53.0  ...         376.0       0.0
..    ...          ...  ...           ...       ...
114  50.0        106.0  ...         645.0       2.0
115  47.0        119.0  ...         620.0       2.0
116  45.0        107.0  ...         610.0       2.0
117  47.0        118.0  ...         650.0       2.0
118  50.0        120.0  ...         605.0       2.0

[119 rows x 30 columns]>

In [ ]:
# Once we load in the file, lets save 20% of our data for testing.
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)
y_train = keras.utils.to_categorical(y_train) #turning targets into categorical vectors training
y_test = keras.utils.to_categorical(y_test) #turning targets into categorical vectors testing
num_classes = len(y_test[0])

feature_vector_length = len(properties)
input_shape = (feature_vector_length,)

In [ ]:
# Configure the model
c_model = keras.Sequential()
c_model.add(keras.layers.Dense(32,input_shape=input_shape,activation=tf.nn.relu,kernel_initializer='he_uniform'))
c_model.add(keras.layers.Dense(32,activation=tf.nn.relu,kernel_initializer='he_uniform'))
c_model.add(keras.layers.Dense(num_classes,activation=tf.nn.softmax))

In [ ]:
# Compile the model
c_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# Train the model
c_model.fit(X_train,y_train,epochs=64,verbose=1,validation_split=0.2)

Epoch 1/64
3/3 [==============================] - 1s 131ms/step - loss: 718.7652 - accuracy: 0.2368 - val_loss: 679.9272 - val_accuracy: 0.1579
Epoch 2/64
3/3 [==============================] - 0s 17ms/step - loss: 572.7441 - accuracy: 0.2368 - val_loss: 521.0085 - val_accuracy: 0.1579
Epoch 3/64
3/3 [==============================] - 0s 18ms/step - loss: 430.3303 - accuracy: 0.2368 - val_loss: 395.9594 - val_accuracy: 0.1579
Epoch 4/64
3/3 [==============================] - 0s 16ms/step - loss: 327.8615 - accuracy: 0.2368 - val_loss: 303.3656 - val_accuracy: 0.0000e+00
Epoch 5/64
3/3 [==============================] - 0s 15ms/step - loss: 254.8469 - accuracy: 0.1053 - val_loss: 234.3738 - val_accuracy: 0.0526
Epoch 6/64
3/3 [==============================] - 0s 14ms/step - loss: 209.9508 - accuracy: 0.1053 - val_loss: 173.1193 - val_accuracy: 0.0526
Epoch 7/64
3/3 [==============================] - 0s 15ms/step - loss: 163.4439 - accuracy: 0.3289 - val_loss: 145.2840 - val_accuracy: 0

In [ ]:
test_results = c_model.evaluate(X_test,y_test,verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')

# Save the model if it is 100% accurate
if test_results[1]*100 == 100:
    c_model.save('models')

1/1 [==============================] - 0s 19ms/step - loss: 1.6611 - accuracy: 0.7083
Test results - Loss: 1.661119818687439 - Accuracy: 70.83333134651184%


In [ ]:
farmer_00 = [17,50,70,55,66,27,55,100,0,0,48,3.8,0,84,2,0,0,1,0,1799,40,20,15,25,10,699,75,1.4,390,0]
farmer_01 = [24,63,81,65,69,76,75,169,0,109,86,4.2,0.53,102,3,2.5,2.5,2,1.5,1800,52,34,19,55,12,999,124,2,415,0]
farmer_02 = [28,85,84,130,84,57,100,192,0.21,200,111,6.3,1.8,90,7,0,0,0,0,2300,59,40,39,0,0,0,0,0,0,1]
farmer_03 = [30,92,91,165,99,42,158,208,0.25,398,124,6.5,2.8,89,9,0,0,7,0,2600,62,40,40,0,0,0,0,0,0,2]
farmer_04 = [45,106,105,182,130,31,176,256,0.38,520,130,9.5,3.8,80,12,0,0,9,0,3400,69,48,51,101,24,3025,147,3.4,640,2]

t = [farmer_00,farmer_01,farmer_02,farmer_03,farmer_04]
res = c_model.predict(t)

for index,result in enumerate(res):
    print("Farmer ",index)
    print(res[index])
    print("-----------------------------------------------------------------------")

Farmer  0
[5.3027584e-06 9.9999440e-01 3.4188747e-07 0.0000000e+00 0.0000000e+00]
-----------------------------------------------------------------------
Farmer  1
[4.2877885e-16 9.9999928e-01 6.6604457e-07 0.0000000e+00 2.0012070e-17]
-----------------------------------------------------------------------
Farmer  2
[6.3260600e-13 1.6387900e-28 1.7384134e-03 9.9826163e-01 0.0000000e+00]
-----------------------------------------------------------------------
Farmer  3
[1.1079708e-25 0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00]
-----------------------------------------------------------------------
Farmer  4
[0. 0. 0. 0. 1.]
-----------------------------------------------------------------------


In [ ]:
for i,rr in enumerate(res):
    max_i = 0
    max_r = 0
    print("Farmer ",i)
    if i == 0:
        print("True= Malnourished")
    elif i == 1:
        print("True= Normal")
    elif i == 2:
        print("True= Borderline Risk")
    elif i == 3:
        print("True= Overweight At Risk")
    elif i == 4:
        print("True= High Risk")
    for ri,r in enumerate(rr):
        if r > max_r:
            max_i = ri
            max_r = r
    if max_i == 0:
        print("Pred= Malnourished")
    elif max_i == 1:
        print("Pred= Normal")
    elif max_i == 2:
        print("Pred= Borderline Risk")
    elif max_i == 3:
        print("Pred= Overweight At Risk")
    elif max_i == 4:
        print("Pred= High Risk")
    print("-------------------------")

Farmer  0
True= Malnourished
Pred= Normal
-------------------------
Farmer  1
True= Normal
Pred= Normal
-------------------------
Farmer  2
True= Borderline Risk
Pred= Overweight At Risk
-------------------------
Farmer  3
True= Overweight At Risk
Pred= Overweight At Risk
-------------------------
Farmer  4
True= High Risk
Pred= High Risk
-------------------------


In [ ]:
health_status = []

malnourished = []
maln_index = 0
normal = []
norm_index = 1
borderline_risk = []
border_index = 2
overweight_risk = []
overw_index = 3
high_risk = []
high_index = 4

res = c_model.predict(X)

for i,rr in enumerate(res):
    max_i = 0
    max_r = 0
    for ri,r in enumerate(rr):
        if r > max_r:
            max_i = ri
            max_r = r
    if max_i == 0:
        malnourished.append(list(X.loc[i]))
    elif max_i == 1:
        normal.append(list(X.loc[i]))
    elif max_i == 2:
        borderline_risk.append(list(X.loc[i]))
    elif max_i == 3:
        overweight_risk.append(list(X.loc[i]))
    elif max_i == 4:
        high_risk.append(list(X.loc[i]))
health_status.append(malnourished)
health_status.append(normal)
health_status.append(borderline_risk)
health_status.append(overweight_risk)
health_status.append(high_risk)        

**Prediabetic or Diabetic**

In [ ]:
properties = list(df.columns.values)
properties.remove('Diabetic')

y_diabetes = df['Diabetic']

In [ ]:
# Splitting test and training sets
Xdiabetes_train, Xdiabetes_test, ydiabetes_train, ydiabetes_test = train_test_split(X,y_diabetes,test_size=0.2,random_state=0)
ydiabetes_train = keras.utils.to_categorical(ydiabetes_train) #turning targets into categorical vectors training
ydiabetes_test = keras.utils.to_categorical(ydiabetes_test) #turning targets into categorical vectors testing
num_classes = len(ydiabetes_test[0])

feature_vector_length = len(properties)
input_shape = (feature_vector_length,)

In [ ]:
diabetes_model = keras.Sequential()
diabetes_model.add(keras.layers.Dense(32,input_shape=input_shape,activation=tf.nn.relu,kernel_initializer='he_uniform'))
diabetes_model.add(keras.layers.Dense(32,activation=tf.nn.relu,kernel_initializer='he_uniform'))
diabetes_model.add(keras.layers.Dense(num_classes,activation=tf.nn.softmax))

diabetes_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history_diabetes = diabetes_model.fit(Xdiabetes_train,ydiabetes_train,epochs=64,verbose=1,validation_split=0.2)

Epoch 1/64
3/3 [==============================] - 1s 101ms/step - loss: 312.1526 - accuracy: 0.4868 - val_loss: 193.8698 - val_accuracy: 0.5789
Epoch 2/64
3/3 [==============================] - 0s 13ms/step - loss: 233.8617 - accuracy: 0.4868 - val_loss: 133.1479 - val_accuracy: 0.5789
Epoch 3/64
3/3 [==============================] - 0s 13ms/step - loss: 162.1277 - accuracy: 0.4737 - val_loss: 73.9195 - val_accuracy: 0.5789
Epoch 4/64
3/3 [==============================] - 0s 16ms/step - loss: 89.7336 - accuracy: 0.4605 - val_loss: 40.8343 - val_accuracy: 0.4737
Epoch 5/64
3/3 [==============================] - 0s 13ms/step - loss: 65.4732 - accuracy: 0.3684 - val_loss: 69.1991 - val_accuracy: 0.3684
Epoch 6/64
3/3 [==============================] - 0s 13ms/step - loss: 84.1327 - accuracy: 0.4211 - val_loss: 59.1254 - val_accuracy: 0.3684
Epoch 7/64
3/3 [==============================] - 0s 15ms/step - loss: 59.3324 - accuracy: 0.4605 - val_loss: 6.8474 - val_accuracy: 0.5789
Epoch 8/

In [ ]:
test_results_diabetes = diabetes_model.evaluate(Xdiabetes_test,ydiabetes_test,verbose=1)
print(f'Test results - Loss: {test_results_diabetes[0]} - Accuracy: {test_results_diabetes[1]*100}%')

1/1 [==============================] - 0s 20ms/step - loss: 2.7092 - accuracy: 0.7500
Test results - Loss: 2.7092361450195312 - Accuracy: 75.0%


In [ ]:
# Save the model if it is 100% accurate
if test_results_diabetes[1]*100 == 100:
    diabetes_model.save('models')

In [ ]:
farmer_00 = [17,50,70,55,66,27,55,100,0,0,48,3.8,0,84,2,0,0,1,0,1799,40,20,15,25,10,699,75,1.4,390,0]
farmer_01 = [24,63,81,65,69,76,75,169,0,109,86,4.2,0.53,102,3,2.5,2.5,2,1.5,1800,52,34,19,55,12,999,124,2,415,1]
farmer_02 = [28,85,84,130,84,57,100,192,0.21,200,111,6.3,1.8,90,7,0,0,0,0,2300,59,40,39,0,0,0,0,0,0,2]
farmer_03 = [30,92,91,165,99,42,158,208,0.25,398,124,6.5,2.8,89,9,0,0,7,0,2600,62,40,40,0,0,0,0,0,0,3]
farmer_04 = [45,106,105,182,130,31,176,256,0.38,520,130,9.5,3.8,80,12,0,0,9,0,3400,69,48,51,101,24,3025,147,3.4,640,4]

res_diabetic = diabetes_model.predict([farmer_00,farmer_01,farmer_02,farmer_03,farmer_04])

In [ ]:
#print(res_diabetic)

for i,result in enumerate(res_diabetic):
    print("Farmer ",i)
    max_r = 0
    max_i = 0
    for ir,r in enumerate(result):
        if r > max_r:
            max_r = r
            max_i = ir
    if i == 0:
        print("True: Not diabetic")
    elif i == 1:
        print("True: Not diabetic")
    elif i == 2:
        print("True: Not diabetic")
    elif i == 3:
        print("True: Diabetic")
    elif i == 4:
        print("True: Diabetic")
    if max_i == 0:
        print("Pred: Not diabetic")
    elif max_i == 1:
        print("Pred: Prediabetic")
    else:
        print("Pred: Diabetic")
    print("------------------")

Farmer  0
True: Not diabetic
Pred: Not diabetic
------------------
Farmer  1
True: Not diabetic
Pred: Not diabetic
------------------
Farmer  2
True: Not diabetic
Pred: Prediabetic
------------------
Farmer  3
True: Diabetic
Pred: Diabetic
------------------
Farmer  4
True: Diabetic
Pred: Diabetic
------------------


**Recommendation System**

In [ ]:
#import external libraries and functions
import sklearn
from sklearn.neighbors import NearestNeighbors

In [ ]:
#load dataset from CSV file and show first 5 records
food_bank = pd.read_csv('food.csv')
   
food_bank.columns = food_bank.columns
food_bank.head()

,Category,Description,Nutrient Data Bank Number,Data.Alpha Carotene,Data.Ash,Data.Beta Carotene,Data.Beta Cryptoxanthin,Data.Carbohydrate,Data.Cholesterol,Data.Choline,Data.Fiber,Data.Kilocalories,Data.Lutein and Zeaxanthin,Data.Lycopene,Data.Manganese,Data.Niacin,Data.Pantothenic Acid,Data.Protein,Data.Refuse Percentage,Data.Retinol,Data.Riboflavin,Data.Selenium,Data.Sugar Total,Data.Thiamin,Data.Water,Data.Fat.Monosaturated Fat,Data.Fat.Polysaturated Fat,Data.Fat.Saturated Fat,Data.Fat.Total Lipid,Data.Household Weights.1st Household Weight,Data.Household Weights.1st Household Weight Description,Data.Household Weights.2nd Household Weight,Data.Household Weights.2nd Household Weight Description,Data.Major Minerals.Calcium,Data.Major Minerals.Copper,Data.Major Minerals.Iron,Data.Major Minerals.Magnesium,Data.Major Minerals.Phosphorus,Data.Major Minerals.Potassium,Data.Major Minerals.Sodium,Data.Major Minerals.Zinc,Data.Vitamins.Vitamin A - IU,Data.Vitamins.Vitamin A - RAE,Data.Vitamins.Vitamin B12,Data.Vitamins.Vitamin B6,Data.Vitamins.Vitamin C,Data.Vitamins.Vitamin E,Data.Vitamins.Vitamin K
0,BUTTER,"BUTTER,WITH SALT",1001,0,2.11,158,0,0.06,215,19,0.0,717,0,0,0.000,0.042,0.110,0.85,0,671,0.034,1.0,0.06,0.005,15.87,21.021,3.043,51.368,81.11,227.00,1 cup,14,1 tbsp,24,0.000,0.02,2,24,24,576,0.09,2499,684,0.17,0.003,0.0,2.32,7.0
1,BUTTER,"BUTTER,WHIPPED,WITH SALT",1002,0,2.11,158,0,0.06,219,19,0.0,717,0,0,0.004,0.042,0.110,0.85,0,671,0.034,1.0,0.06,0.005,15.87,23.426,3.012,50.489,81.11,151.00,1 cup,9,1 tbsp,24,0.016,0.16,2,23,26,827,0.05,2499,684,0.13,0.003,0.0,2.32,7.0
2,BUTTER OIL,"BUTTER OIL,ANHYDROUS",1003,0,0.00,193,0,0.00,256,22,0.0,876,0,0,0.000,0.003,0.010,0.28,0,824,0.005,0.0,0.00,0.001,0.24,28.732,3.694,61.924,99.48,205.00,1 cup,13,1 tbsp,4,0.001,0.00,0,3,5,2,0.01,3069,840,0.01,0.001,0.0,2.80,8.6
3,CHEESE,"CHEESE,BLUE",1004,0,5.11,74,0,2.34,75,15,0.0,353,0,0,0.009,1.016,1.729,21.40,0,192,0.382,14.5,0.50,0.029,42.41,7.778,0.800,18.669,28.74,28.35,1 oz,17,1 cubic inch,528,0.040,0.31,23,387,256,1395,2.66,763,198,1.22,0.166,0.0,0.25,2.4
4,CHEESE,"CHEESE,BRICK",1005,0,3.18,76,0,2.79,94,15,0.0,371,0,0,0.012,0.118,0.288,23.24,0,286,0.351,14.5,0.51,0.014,41.11,8.598,0.784,18.764,29.68,132.00,"1 cup, diced",113,"1 cup, shredded",674,0.024,0.43,24,451,136,560,2.60,1080,292,1.26,0.065,0.0,0.26,2.5


In [ ]:
data_key = pd.read_csv('mock_data_key.csv')

print(data_key)

                            MALNOURISHED  ... HIGH RISK
BMI                                 < 19  ...      ≥ 40
Weight (Kg)                         < 63  ...      ≥ 98
Waist Circumference (cm)            ≤ 73  ...     ≥ 103
Systolic BP (mmHg)                  ≤ 60  ...     ≥ 181
Diastolic BP (mmHg)                 ≤ 67  ...     ≥ 121
HDL-c (mg/dL)                       < 30  ...      ≤ 40
LDL-c (mg/dL)                       < 65  ...     > 160
Total Cholesterol (mg/dL)          < 110  ...     ≥ 240
Atherogenicity index                   X  ...     > 0.3
TAG (mg/dL)                            X  ...     ≥ 500
FPG (mg/dL)                         < 75  ...     > 125
HbA1c (%)                            < 4  ...     ≥ 8.0
CRP (mg/L)                             X  ...     > 3.0
Pulse Oximetry (%)                  < 85  ...      < 85
Servings Grain (oz.)               < 3.0  ...      > 10
Servings Fruit (2c.)                 0-1  ...         X
Servings Veg (2.5c.)                 0-1  ...   

In [ ]:
farmer_00 = [17,50,70,55,66,27,55,100,0,0,48,3.8,0,84,2,0,0,1,0,1799,40,20,15,25,10,699,75,1.4,390,0]

# Indices to where each farmer's vitamin measures are
v_e = 24
v_a = 25
v_k = 26
v_b12 = 27

In [ ]:
# Food bank indices of vitamins we want
f_a_index = 41
f_b12_index = 43
f_e_index = 46
f_k_index = 47


In [ ]:
# Recommended Vitamins
rec_vitamins = {}
rec_vitamins["vitamin_e"] = 12
rec_vitamins["vitamin_a"] = 700
rec_vitamins["vitamin_k"] = 110
rec_vitamins["vitamin_b12"] = 1.5

recommendations = []


In [ ]:
# Input: A list of vitamins that might be needed
# Outputs: A dictionary of the vitamin name as key and the amount of vitamin needed as value
def within_range(need_list):
    threshold = 5
    vitamins_needed = {}

    vitamins_needed["Vitamin_E"] = 0
    vitamins_needed["Vitamin_A"] = 0
    vitamins_needed["Vitamin_K"] = 0
    vitamins_needed["Vitamin_B12"] = 0
    
    # If a person only need 5 or less units of vitamin, we don't recommend anything
    for ni,n in enumerate(need_list):
        if n > threshold:
            vitamin = ""
            if ni == 0:
                vitamins_needed["Vitamin_E"] = n
            elif ni == 1:
                vitamins_needed["Vitamin_A"] = n
            elif ni == 2:
                vitamins_needed["Vitamin_K"] = n
            elif ni == 3:
                vitamins_needed["Vitamin_B12"] = n
    return vitamins_needed

In [ ]:
# Input: 
# Output:
def get_food_rec(farmer):
    # Get the farmer's needed vitamins
    need_vitamin_e = rec_vitamins["vitamin_e"] - farmer[v_e]
    need_vitamin_a = rec_vitamins["vitamin_a"] - farmer[v_a]
    need_vitamin_k = rec_vitamins["vitamin_k"] - farmer[v_k]
    need_vitamin_b12 = rec_vitamins["vitamin_b12"] - farmer[v_b12]
    
    need_list = []
    need_list.append(need_vitamin_e)
    need_list.append(need_vitamin_a)
    need_list.append(need_vitamin_k)
    need_list.append(need_vitamin_b12)

    vitamins_needed = within_range(need_list)
    
    X_rec = food_bank.iloc[:,[f_e_index,f_a_index,f_k_index,f_b12_index]].values
    nbrs = NearestNeighbors(n_neighbors=1).fit(X_rec)
    
    foods = {}
    food_quantity = {}
    
    not_in_range = True
    for key in vitamins_needed:
        if vitamins_needed[key] != 0:
            food_list = []
            if key == "Vitamin_E":
                while vitamins_needed[key] > 5:
                    food = food_bank.iloc[nbrs.kneighbors([[vitamins_needed["Vitamin_E"],0,0,0]])[1][0][0]]
                    if food[0] not in food_quantity:
                        food_quantity[food[0]] = 1
                        food_list.append(food)
                     else:
                        food_quantity[food[0]] += 1
                    vitamins_needed[key] -= food[f_e_index]
                foods["Vitamin_E"] = food_list
            elif key == "Vitamin_A":
                while vitamins_needed[key] > 5:
                    food = food_bank.iloc[nbrs.kneighbors([[0,vitamins_needed["Vitamin_A"],0,0]])[1][0][0]]
                    if food[0] not in food_quantity:
                        food_quantity[food[0]] = 1
                        food_list.append(food)
                    else:
                        food_quantity[food[0]] += 1
                    vitamins_needed[key] -= food[f_a_index]
                foods["Vitamin_A"] = food_list
            elif key == "Vitamin_K":
                while vitamins_needed[key] > 5:
                    food = food_bank.iloc[nbrs.kneighbors([[0,0,vitamins_needed["Vitamin_K"],0]])[1][0][0]]
                    if food[0] not in food_quantity:
                        food_quantity[food[0]] = 1
                        food_list.append(food)
                    else:
                        food_quantity[food[0]] += 1
                    vitamins_needed[key] -= food[f_k_index]
                foods["Vitamin_K"] = food_list
            elif key == "Vitamin_B12":
                while vitamins_needed[key] > 5:
                    food = food_bank.iloc[nbrs.kneighbors([[0,0,0,vitamins_needed["Vitamin_B12"]]])[1][0][0]]
                    if food[0] not in food_quantity:
                        food_quantity[food[0]] = 1
                        food_list.append(food)
                    else:
                        food_quantity[food[0]] += 1
                    vitamins_needed[key] -= food[f_b12_index]
                foods["Vitamin_B12"] = food_list
    return foods,food_quantity
def print_rec(foods,food_quantity):
    for k in foods:
        print("For " + k + " we recommend:")
        for food in foods[k]:
            print(food[0] + " x" + str(food_quantity[food[0]]) + " serving(s)")
        print("-------------------------------")
        
foods,food_quantity = get_food_rec(farmer_00)
print_rec(foods,food_quantity)    

IndentationError: ignored

In [ ]:
n_farmer = health_status[norm_index][0]
n_food,food_q = get_food_rec(n_farmer)
print_rec(n_food,food_q)

NameError: ignored